# Section 1-0 - Prelude

We'll start with the Kaggle Titanic dataset. The dataset is a list of Titanic passengers with features such as class, age, sex and fare. We'll use these features to train a model, and use the model to predict whether or not each passenger survived. A more detailed treatment of the dataset can be found here:

https://github.com/savarin/python_for_ml

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('data/titanic.csv')

In [2]:
df.head()

,Survived,Class,Sex,Age,Fare
0,0,3,1,22,7.2500
1,1,1,0,38,71.2833
2,1,3,0,26,7.9250
3,1,1,0,35,53.1000
4,0,3,1,35,8.0500


We split the data 80-20 into training and test sets. In addition, we scale the data so that each column has mean 0 and standard deviation 1, and create one-hot vectors with the labels (analogous to dummy variables).

In [3]:
df_train = df.iloc[:712, :]

scaler = StandardScaler()
features = ['Class', 'Sex', 'Age', 'Fare']

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

In [4]:
X_train[:5, :]

array([[ 0.83290956,  0.74926865, -0.61259594, -0.51933199],
       [-1.55353553, -1.33463478,  0.6184268 ,  0.79718222],
       [ 0.83290956, -1.33463478, -0.30484025, -0.5054541 ],
       [-1.55353553, -1.33463478,  0.38761004,  0.42333654],
       [ 0.83290956,  0.74926865,  0.38761004, -0.50288412]])

In [5]:
y_train[:5]

array([0, 1, 1, 1, 0])

In [6]:
y_train_onehot[:5]

array([[ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.]])

In [7]:
df_test = df.iloc[712:, :]

X_test = scaler.transform(df_test[features].values)
y_test = df_test['Survived'].values

 ## Benchmark

To create a basis for comparison, we train a Random Forest model and record the accuracy on the test set.

In [8]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, y_train)

y_prediction = model.predict(X_test)
print "\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

accuracy 0.810055865922


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


## 1-layer Neural Network

For the input, we have a vector of length 4 that represents each passenger's features. As an example, we consider the first passenger in the dataset.

In [9]:
print X_train[0]

[ 0.83290956  0.74926865 -0.61259594 -0.51933199]


For the output, we want a vector of length 2 to represent the survival probabilities. A simple way to create this mapping is by using a 2x4 matrix. To start off, we generate a random matrix representing feature weights and apply the matrix to our input. We'll also add a bias term.

In [10]:
W = np.random.rand(2, 4) * 0.01

print W

[[ 0.00262025  0.00158684  0.00278127  0.00459317]
 [ 0.00321001  0.00518393  0.00261943  0.00976085]]


In [11]:
b = np.random.rand(2,) * 0.01

print b

[ 0.00732815  0.00115274]


In [12]:
result = np.dot(W, X_train[0]) + b

print result

[ 0.00661037  0.00103677]


To get the output vector to sum to 1, we apply a softmax mapping. The first element would now represent the probability that the passenger did not survive, and the second element represents the probability the passenger survives.

In [13]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum()

In [14]:
result = softmax(result)

print result

[ 0.5013934  0.4986066]


We can then compare the output vector to the actual label. We would have a 'good' model if the probability for the correct label was close to 1, and a 'bad' one if it was close to 0.

In [15]:
print y_train_onehot[0]

[ 1.  0.]


In [16]:
label_index = np.argmax(y_train_onehot[0])

print label_index

0


In [17]:
print "predicted label-0 probability", result[label_index]

predicted label-0 probability 0.501393397782


We define loss to be the negative logarithm of the probability of the correct label. Taking the logarithm penalizes the model for having a high probability associated with the wrong label. Here we have the loss associated with the first passenger.

In [18]:
loss = -np.log(result[label_index])

print "loss for first passenger", loss

loss for first passenger 0.690364260912


We run the calculation through all the passengers in the training set, and divide the total loss by the number of passengers to obtain the average loss.

In [19]:
for j in xrange(X_train.shape[0]):
    result = np.dot(W, X_train[j]) + b
    result = softmax(result)
        
    label_index = np.argmax(y_train_onehot[j])
    loss += -np.log(result[label_index])

loss = loss / float(X_train.shape[0])

print "average loss across all passengers", loss

average loss across all passengers 0.693894042507


Now we iterate through 1000 iterations for random values of W and b, and keep the pair which minimizes the average loss.

In [20]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W = np.random.rand(2, 4) / 10
    b = np.random.rand(2,) / 10

    scores = []
    loss = 0
    
    for j in xrange(X_train.shape[0]):
        result = np.dot(W, X_train[j]) + b
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])

    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))
    
    if loss < min_loss:
        min_loss = loss
        best_weights = (W, b)
        print "loss %s accuracy %s loop %s" % (round(loss, 3), round(accuracy, 3), i)

print "\ntime taken %s seconds" % str(time() - start)

loss 0.711 accuracy 0.317 loop 0
loss 0.698 accuracy 0.393 loop 1
loss 0.67 accuracy 0.772 loop 2
loss 0.663 accuracy 0.785 loop 4
loss 0.661 accuracy 0.749 loop 156
loss 0.659 accuracy 0.725 loop 452
loss 0.658 accuracy 0.787 loop 715

time taken 8.82983803749 seconds


In [21]:
W, b = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W, X_test[j]) + b
    result = softmax(result)
    scores.append(list(result))

y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.782122905028


For each passenger, predictions for the test set were made by selecting the label with the highest probability. Despite the naïve approach, we obtain a prediction accuracy of 78%!

## 2-layer Neural Network

With the 1-layer neural network, we had a 2x4 weight matrix. To create more degrees of freedom, we can introduce intermediary matrices or 'layers'. For example, instead of having a mapping from a vector of length 4 to a vector of length 2, we'll have two mappings - first from 4 to 100, followed by 100 to 2.

In [22]:
W_1 = np.random.rand(100, 4) * 0.01
b_1 = np.random.rand(100,) * 0.01
W_2 = np.random.rand(2, 100) * 0.01
b_2 = np.random.rand(2,) * 0.01

In [23]:
result = np.dot(W_1, X_train[0]) + b_1

print result

[  1.13174863e-02   3.27517674e-03   6.64636413e-03   6.67664294e-03
   3.43098480e-03   1.36640330e-02   9.18247683e-03   1.17094685e-02
   1.01565364e-03  -1.60432219e-03   9.35207856e-03   7.47293230e-03
   1.52676682e-02   7.97936749e-03   6.55563928e-04   6.00362175e-03
   7.23496560e-03   9.92625256e-03   4.14720990e-03   5.41832887e-03
   1.18845724e-02   1.60057818e-02   7.64679793e-03   6.94767477e-03
   1.33371235e-02   6.07771410e-03   1.96233598e-02   1.57180776e-02
   9.84855859e-03   3.82086941e-03   3.77595128e-04   2.26082278e-03
   1.45080114e-03   1.45319158e-02   7.90452470e-03   8.92775565e-03
   1.61055072e-02   8.93604299e-03   1.02217040e-02   9.01988122e-03
   1.02962602e-02   4.56788321e-03   1.14912002e-02  -2.16470496e-04
   7.17207370e-03   1.32500054e-02   2.73586936e-03   4.19898797e-03
   5.70306531e-03   9.32144683e-03   8.41222049e-03   1.03885822e-02
   1.64928832e-02  -2.05164035e-03   1.72807609e-02   3.94223376e-03
  -3.86111353e-03   1.54912269e-02

In [24]:
result = np.dot(W_2, result) + b_2

print result

[ 0.00977321  0.00541989]


As before we iterate through 1000 iterations of random values for W_1, b_1, W_2 and b_2, and keep the one which minimizes loss.

In [25]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W_1 = np.random.rand(100, 4) * 0.01
    b_1 = np.random.rand(100,) * 0.01
    W_2 = np.random.rand(2, 100) * 0.01
    b_2 = np.random.rand(2,) * 0.01
    
    scores = []
    loss = 0

    for j in xrange(X_train.shape[0]):
        result = np.dot(W_1, X_train[j]) + b_1
        result = np.dot(W_2, result) + b_2
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])

    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))
        
    if loss < min_loss:
        min_loss = loss
        best_weights = (W_1, b_1, W_2, b_2)
        print "loss %s accuracy %s loop %s" % (round(loss, 3), round(accuracy, 3), i)

print "\ntime taken %s seconds" % str(time() - start)

loss 0.693 accuracy 0.375 loop 0
loss 0.693 accuracy 0.61 loop 1
loss 0.692 accuracy 0.61 loop 3
loss 0.692 accuracy 0.61 loop 22
loss 0.692 accuracy 0.61 loop 59
loss 0.692 accuracy 0.61 loop 144
loss 0.692 accuracy 0.61 loop 229
loss 0.692 accuracy 0.61 loop 511
loss 0.692 accuracy 0.61 loop 808
loss 0.692 accuracy 0.61 loop 809

time taken 10.5175900459 seconds


In [26]:
W_1, b_1, W_2, b_2 = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W_1, X_test[j]) + b_1
    result = np.dot(W_2, result) + b_2
    result = softmax(result)
    scores.append(list(result))
    
y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.642458100559


Despite the greater degree of freedom, we get an accuracy score of only 64%.

## 3-layer Neural Network

We take this a step further by adding an additional layer, and similarly review model performance.

In [27]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W_1 = np.random.rand(100, 4) * 0.01
    b_1 = np.random.rand(100,) * 0.01
    W_2 = np.random.rand(100, 100) * 0.01
    b_2 = np.random.rand(100,) * 0.01
    W_3 = np.random.rand(2, 100) * 0.01
    b_3 = np.random.rand(2,) * 0.01
    
    scores = []
    loss = 0

    for j in xrange(X_train.shape[0]):
        result = np.dot(W_1, X_train[j]) + b_1
        result = np.dot(W_2, result) + b_2
        result = np.dot(W_3, result) + b_3
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])
        
    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))          
        
    if loss < min_loss:
        min_loss = loss
        best_weights = (W_1, b_1, W_2, b_2, W_3, b_3)
        print "loss %s accuracy %s loop %s" % (round(loss, 3), round(accuracy, 3), i)

print "\ntime taken %s seconds" % str(time() - start)

loss 0.693 accuracy 0.61 loop 0
loss 0.693 accuracy 0.61 loop 1
loss 0.693 accuracy 0.61 loop 20
loss 0.692 accuracy 0.61 loop 58
loss 0.692 accuracy 0.61 loop 74
loss 0.692 accuracy 0.61 loop 563
loss 0.692 accuracy 0.61 loop 809
loss 0.692 accuracy 0.61 loop 990

time taken 13.771780014 seconds


In [28]:
W_1, b_1, W_2, b_2, W_3, b_3 = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W_1, X_test[j]) + b_1
    result = np.dot(W_2, result) + b_2
    result = np.dot(W_3, result) + b_3    
    result = softmax(result)
    scores.append(list(result))
    
y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.642458100559


We run into the same problem of suboptimal model performance, but this is not unexpected given the simplistic approach taken for illustrative purposes. We look to build on this in the next section by taking a systematic approach to optimizing the weight matrices and biases.